In [33]:
import pandas as pd
import time
import csv

from tqdm.auto import tqdm

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key="gQK7DGi9o3fLJrl97qyrVGfBh28QP3FO",
    base_url="https://api.upstage.ai/v1/solar"
)

In [3]:
# this system prompt will be used globally on the chat session.
system_prompt = {
    "role": "system",
    "content": "You are a helpful assistant."
}

In [4]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
df_dev = pd.read_csv("../data/dev.csv")

In [5]:
df_train

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [6]:
df_dev

,fname,dialogue,summary,topic
0,dev_0,"#Person1#: 안녕하세요, 오늘 하루 어떠셨어요? \n#Person2#: 요즘...",#Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대...,의사에게 상담하기
1,dev_1,"#Person1#: 헤이, 지미. 나중에 운동하러 가자.\n#Person2#: 그래...",#Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.,운동하기
2,dev_2,#Person1#: 나는 더 이상 건강에 해로운 음식을 먹는 것을 멈춰야 해.\n#...,"#Person1#은 건강에 해로운 음식을 먹는 것을 멈추려는 계획을 세우고, #Pe...",건강한 음식
3,dev_3,"#Person1#: UFO를 믿으세요?\n#Person2#: 물론이죠, 그들은 저기...",#Person2#는 UFO를 믿고 꿈에서 그들을 볼 수 있다고 말한다. #Perso...,UFO와 외계인
4,dev_4,#Person1#: 오늘 학교에 갔어?\n#Person2#: 당연하지. 너는?\n#...,#Person1#은 오늘 학교에 가지 않았다. #Person2#는 내일 수업을 빼먹...,학교 가기
...,...,...,...,...
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#P...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2...,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#...,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동...",#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교...,소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지...,빠진 페이지


In [7]:
df_test

,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, ..."
1,test_1,#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또...
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Per..."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person..."
4,test_4,#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금...
...,...,...
494,test_495,"#Person1#: 헤이, 찰리, 학교 끝나고 우리 집에 와서 나랑 비디오 게임 할..."
495,test_496,#Person1#: 어떻게 컨트리 음악에 관심을 가지게 되었나요?\r\n#Perso...
496,test_497,"#Person1#: 실례합니다, 앨리스. 이곳을 사용해본 적이 없는데, 기계를 어떻..."
497,test_498,#Person1#: 매튜? 안녕!\r\n#Person2#: 스티브! 오랜만이네! 얼...


In [8]:
df_train.iloc[0]

fname                                                 train_0
dialogue    #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...
summary     스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...
topic                                                 건강검진 받기
Name: 0, dtype: object

In [9]:
data = list(zip(df_train["fname"], df_train["dialogue"], df_train["summary"], df_train["topic"]))

In [13]:
model = "solar-1-mini-chat"

dialogue_num = 1

# text_summary_prompt = f"내가 예시로 주는 대화문을 보고 대화문의 요약 예시와 유사하게 요약이 할 수 있도록 해당 대화문을 재구성해서 대화문을 {dialogue_num}개 만들어줘"
text_summary_prompt = f'내가 위에 준 대화문 예시와 동일한 주제로 유사한 새로운 데이터를 \n"대화문 :"\n"요약문 :"\n"주제 :" 양식으로 맞춰서 만들어줘'

make_dialoge = {}

with open("data_aggrement_by_solar.txt", "a", newline="") as f:
    for fname ,dial, summa, topic in tqdm(data):
        chat_history = []
        history_size = 6
        
        for i in range(1, dialogue_num+1):
            # Step 1: Get user input.
            mk_fname = f"{fname}_{i}"
            user_prompt = "\n\n대화문 예시:\n" + dial + "\n요약문 예시:" + summa + "\n주제 예시:" + topic + "\n\n" + text_summary_prompt
            chat_history.append({
                "role": "user",
                "content": user_prompt
            })

            # Step 2: Use the chat history to generate the chatbot's response.
            messages = [system_prompt] + chat_history
            stream = client.chat.completions.create(
                model=model,
                messages=messages,
                stream=True,
            )
            
            # Step 3: Output the Solar response chunks.
            solar_response = ""
            for chunk in stream:
                response_content = chunk.choices[0].delta.content
                if response_content is not None:
                    solar_response += response_content
            # Finish response output.
            
            f.write(mk_fname)
            f.write("\n\n")
            f.write(solar_response)
            f.write("\n\n")

            # Step 4: Append the Solar response to the chat history.
            chat_history.append({
                "role": "assistant",
                "content": solar_response
            })
            time.sleep(3)
            
            

        # Step 5: Ensure the chat history doesn't exceed the size limit.
        chat_history = chat_history[:history_size]

  0%|          | 0/285 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
data = []
context = ""
with open("./data_aggrement_by_solar2.txt", "r") as f:
    for line in f.readlines():
        if "train" not in line:
            context += line
        else :
            data.append(context)
            context = ""

In [13]:
data = data[1:]

In [21]:
data2 = []
for text in data:
    data2.append(text.strip("\n<split>\n").strip("."))

In [27]:
data2

['#Person1#: 안녕하세요, 존스씨. 저는 리 박사입니다. 오늘은 어떤 이유로 오셨나요?\n#Person2#: 사실, 저는 건강검진을 받으러 왔어요.\n#Person1#: 그렇군요. 존스씨, 건강검진을 받는 것은 매우 중요합니다. 어떤 증상이 있으셨나요?\n#Person2#: 아직은 특별한 증상은 없어요. 하지만 건강을 유지하고 싶어서 검진을 받으려고 해요.\n#Person1#: 현명한 선택이십니다. 저희 병원에서는 종합적인 건강검진을 제공하고 있습니다. 혈액검사, X-레이, 내시경 등 다양한 검사를 통해 질병을 조기에 발견하고 예방할 수 있습니다.\n#Person2#: 그렇군요. 저도 건강검진을 받아보고 싶어요. 어떻게 예약을 하면 될까요?\n#Person1#: 좋아요, 오늘 바로 예약해드릴게요. 예약 후에는 검진 전에 준비해야 할 사항과 검진 시간에 대해 안내해드릴게요.\n#Person2#: 감사합니다, 리 박사님. 건강검진을 통해 건강한 삶을 유지하고 싶어요.\n\n<split> 존스씨가 건강검진을 받기 위해 리 박사를 찾았습니다. 리 박사는 건강검진의 중요성을 강조하고, 종합적인 건강검진을 제공하는 병원의 서비스를 설명했습니다. 존스씨는 건강검진을 예약하고, 몸을 건강하게 유지하기 위해 노력하기로 했습니다.\n\n<split> 건강검진 받기',
 '#Person1#: 안녕하세요, 스미스 여사님. 어떻게 지내셨나요?\n#Person2#: 안녕하세요, 존슨 박사님. 잘 지냈어요, 감사합니다. 제니를 예방접종하러 왔어요.\n#Person1#: 좋아요. 제니의 예방접종 기록을 확인해 볼게요. 제니는 이미 DTaP, 소아마비, 흡혈병 예방접종을 받았군요. 이제 18개월이 되었으니, MMR과 A형 간염 예방접종을 받아야 할 때가 되었어요.\n#Person2#: 로타바이러스 예방접종은 어떻게 되나요?\n#Person1#: 로타바이러스 예방접종은 선택적인 예방접종이에요. 이 예방접종은 로타바이러스로 인한 심각한 위장 질환을 예방하는 데 도움이 됩니다.\n#Pe

In [34]:
dialogues = []
summarys = []
topics = []
for text in data2:
    if text.count("<split>") == 2:
        dialogue, summary, topic = text.split("<split>")
        dialogues.append(dialogue)
        summarys.append(summary)
        topics.append(topic)

df_new_data = pd.DataFrame({
    "dialogue" : dialogues,
    "summary" : summarys,
    "topic" : topics
})

In [35]:
df_new_data

,dialogue,summary,topic
0,"#Person1#: 안녕하세요, 존스씨. 저는 리 박사입니다. 오늘은 어떤 이유로 ...",존스씨가 건강검진을 받기 위해 리 박사를 찾았습니다. 리 박사는 건강검진의 중요성...,건강검진 받기
1,"#Person1#: 안녕하세요, 스미스 여사님. 어떻게 지내셨나요?\n#Person...","스미스 여사님이 제니를 예방접종하러 왔고, 존슨 박사님이 제니에게 MMR, A형 ...",예방접종
2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2...",열쇠 찾기
3,#Person1#: 왜 너는 여자친구가 있다는 걸 말하지 않았어?\n#Person2...,#Person1#은 #Person2#가 여자친구가 있다는 사실을 말하지 않아서 화...,여자친구가 있다
4,"#Person1#: 안녕, 친구들! 우리 함께 춤추러 가자! 파티를 즐겨보자!\n#...","#Person1#이 #Person2#에게 춤을 추러 가자고 제안하고, #Perso...",춤
...,...,...,...
12050,"#Person1#: 안녕, 어떻게 지내?\n#Person2#: 별로야. 나는 이 역...",\n#Person1#과 #Person2#는 각각 19세기 유럽에서의 혁명의 영향과 ...,논문 쓰기
12051,"#Person1#: 안녕, 너를 태워줄까?\n#Person2#: 네, 부탁해!\n#...",#Person1#은 #Person2#를 태우고 쇼핑몰로 향하기로 합의했습니다.\n,태우다
12052,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨...,컨퍼런스 센터
12053,#Person1#: 안녕하세요. 오늘은 무엇을 도와드릴까요?\n#Person2#: ...,#Person2#는 #Person1#의 도움으로 7일 동안 중형 차를 렌트합니다.\n,차량 렌트


In [36]:
df_new_data["dialogue"] = df_new_data["dialogue"].str.strip("\n")
df_new_data["dialogue"] = df_new_data["dialogue"].str.strip(" ")
df_new_data["summary"] = df_new_data["summary"].str.strip("\n")
df_new_data["summary"] = df_new_data["summary"].str.strip(" ")
df_new_data["topic"] = df_new_data["topic"].str.strip("\n")
df_new_data["topic"] = df_new_data["topic"].str.strip(" ")

In [38]:
df_new_data.to_csv("solar_using_data_agmentation.csv", index=False)

In [42]:
df_train_bt = pd.read_csv("../data/train_add_backtranslation.csv")

In [43]:
del df_train_bt["fname"]

In [44]:
df_train_bt

,dialogue,summary,topic
0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,"#Person1#: 안녕하세요, 스미스 씨. 저는 호킨스 박사입니다. 오늘은 왜 ...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
2,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
3,"#Person1#: 안녕하세요, 파커 부인, 잘 지내세요? #Person2#: 안...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
4,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
...,...,...,...
24909,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
24910,#Person1#: 오늘 기분이 안 좋아 보이는군요. 무슨 일이야? #Person2...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
24911,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
24912,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [45]:
df_new_data

,dialogue,summary,topic
0,"#Person1#: 안녕하세요, 존스씨. 저는 리 박사입니다. 오늘은 어떤 이유로 ...",존스씨가 건강검진을 받기 위해 리 박사를 찾았습니다. 리 박사는 건강검진의 중요성을...,건강검진 받기
1,"#Person1#: 안녕하세요, 스미스 여사님. 어떻게 지내셨나요?\n#Person...","스미스 여사님이 제니를 예방접종하러 왔고, 존슨 박사님이 제니에게 MMR, A형 간...",예방접종
2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,#Person1#: 왜 너는 여자친구가 있다는 걸 말하지 않았어?\n#Person2...,#Person1#은 #Person2#가 여자친구가 있다는 사실을 말하지 않아서 화가...,여자친구가 있다
4,"#Person1#: 안녕, 친구들! 우리 함께 춤추러 가자! 파티를 즐겨보자!\n#...","#Person1#이 #Person2#에게 춤을 추러 가자고 제안하고, #Person...",춤
...,...,...,...
12050,"#Person1#: 안녕, 어떻게 지내?\n#Person2#: 별로야. 나는 이 역...",#Person1#과 #Person2#는 각각 19세기 유럽에서의 혁명의 영향과 20...,논문 쓰기
12051,"#Person1#: 안녕, 너를 태워줄까?\n#Person2#: 네, 부탁해!\n#...",#Person1#은 #Person2#를 태우고 쇼핑몰로 향하기로 합의했습니다.,태우다
12052,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12053,#Person1#: 안녕하세요. 오늘은 무엇을 도와드릴까요?\n#Person2#: ...,#Person2#는 #Person1#의 도움으로 7일 동안 중형 차를 렌트합니다.,차량 렌트


In [67]:
df_concat_data = pd.concat([df_train_bt, df_new_data], axis=0)

In [68]:
df_concat_data

,dialogue,summary,topic
0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,"#Person1#: 안녕하세요, 스미스 씨. 저는 호킨스 박사입니다. 오늘은 왜 ...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
2,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
3,"#Person1#: 안녕하세요, 파커 부인, 잘 지내세요? #Person2#: 안...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
4,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
...,...,...,...
12050,"#Person1#: 안녕, 어떻게 지내?\n#Person2#: 별로야. 나는 이 역...",#Person1#과 #Person2#는 각각 19세기 유럽에서의 혁명의 영향과 20...,논문 쓰기
12051,"#Person1#: 안녕, 너를 태워줄까?\n#Person2#: 네, 부탁해!\n#...",#Person1#은 #Person2#를 태우고 쇼핑몰로 향하기로 합의했습니다.,태우다
12052,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12053,#Person1#: 안녕하세요. 오늘은 무엇을 도와드릴까요?\n#Person2#: ...,#Person2#는 #Person1#의 도움으로 7일 동안 중형 차를 렌트합니다.,차량 렌트


In [72]:
df_concat_data = df_concat_data.reset_index(drop=True).reset_index()

In [74]:
df_concat_data["index"] = df_concat_data["index"].apply(lambda x : "train_" + str(x))

In [77]:
df_concat_data.rename(columns={"index" : "fname"}, inplace=True)

In [78]:
df_concat_data

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 스미스 씨. 저는 호킨스 박사입니다. 오늘은 왜 ...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
2,train_2,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
3,train_3,"#Person1#: 안녕하세요, 파커 부인, 잘 지내세요? #Person2#: 안...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
4,train_4,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
...,...,...,...,...
36964,train_36964,"#Person1#: 안녕, 어떻게 지내?\n#Person2#: 별로야. 나는 이 역...",#Person1#과 #Person2#는 각각 19세기 유럽에서의 혁명의 영향과 20...,논문 쓰기
36965,train_36965,"#Person1#: 안녕, 너를 태워줄까?\n#Person2#: 네, 부탁해!\n#...",#Person1#은 #Person2#를 태우고 쇼핑몰로 향하기로 합의했습니다.,태우다
36966,train_36966,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
36967,train_36967,#Person1#: 안녕하세요. 오늘은 무엇을 도와드릴까요?\n#Person2#: ...,#Person2#는 #Person1#의 도움으로 7일 동안 중형 차를 렌트합니다.,차량 렌트


In [79]:
df_concat_data.to_csv("new_train.csv", index=False)